In [1]:
import gvar as gv
import numpy as np 
import load_data_priors as ld 
import bs_utils as bs 
import h5py as h5

In [2]:
file = '/home/gmoney/lqcd/data/c51_2pt_octet_decuplet.h5'
abbr = 'a09m135'

raw = ld.get_raw_corr_new(file,abbr)
bs_M = raw['proton_SS'].shape[0] 
bs_N = 10000 # In real-world cases, this would probably be much larger
bs_list = bs.get_bs_list(bs_M,bs_N)


In [3]:
data = {}
gmo = {}
for src_snk in ['PS', 'SS']:
    for baryon in ['lambda_z', 'sigma_p', 'proton', 'xi_z']:
        data[baryon+'_'+src_snk] = raw[baryon+'_'+src_snk][bs_list].mean(axis=0)
        gmo[src_snk] = np.ones_like(data[baryon+'_'+src_snk])
    gmo[src_snk] *= data['lambda_z_'+src_snk] *np.power(data['sigma_p_'+src_snk], 1/3) *np.power(data['proton_'+src_snk], -2/3) * np.power(data['xi_z_'+src_snk], -2/3)

gv_data = gv.dataset.avg_data(gmo,bstrap=True)
# gv_data
print(gv_data)

/tmp/ipykernel_4045503/428993121.py:7: RuntimeWarning: invalid value encountered in power
  gmo[src_snk] *= data['lambda_z_'+src_snk] *np.power(data['sigma_p_'+src_snk], 1/3) *np.power(data['proton_'+src_snk], -2/3) * np.power(data['xi_z_'+src_snk], -2/3)


{'PS': array([0.6462(26), 0.605279(64), 0.60404(27), 0.60245(45), 0.60131(39),
       0.59996(28), 0.59844(27), 0.59710(24), 0.59570(23), 0.59444(26),
       0.59316(33), 0.59188(45), 0.59079(63), 0.58995(81), 0.5881(10),
       0.5850(12), 0.5827(16), 0.5860(21), 0.5879(30), 0.5792(38),
       0.5703(53), 0.5718(70), 0.5791(97), 0.585(13), 0.595(18),
       0.645(33), 0.684(56), 0.686(72), 0.71(12), 0.71(15), 0.545(84),
       0.458(67), 0.419(82), 0.319(57), 0.212(33), nan +- nan, nan +- nan,
       nan +- nan, nan +- nan, nan +- nan, nan +- nan, nan +- nan,
       nan +- nan, nan +- nan, nan +- nan, nan +- nan, nan +- nan,
       nan +- nan, nan +- nan, nan +- nan, nan +- nan, nan +- nan,
       nan +- nan, nan +- nan, nan +- nan, nan +- nan, nan +- nan,
       nan +- nan, nan +- nan, nan +- nan, nan +- nan, nan +- nan,
       nan +- nan, nan +- nan, nan +- nan, nan +- nan, nan +- nan,
       nan +- nan, nan +- nan, nan +- nan, nan +- nan, nan +- nan,
       nan +- nan, nan +- nan, 

In [4]:
model_type = 'gmo_direct'
import os 
import sys
import importlib
import gmo_fit_analysis as fa 
fit_params = 'input_files/a09m135.py'
sys.path.append(os.path.dirname(os.path.abspath(fit_params)))
fp = importlib.import_module(
        fit_params.split('/')[-1].split('.py')[0])


p_dict = fp.p_dict
abbr = p_dict['abbr']
prior = ld.fetch_prior(model_type,p_dict)
print(prior)
gmo_direct = fa.fit_analysis(t_range=p_dict
['t_range'],simult=False,t_period=64,states=['gmo'],p_dict=p_dict, 
n_states=p_dict['n_states'],prior=prior,
nucleon_corr_data=None,lam_corr_data=None, xi_corr_data=None,sigma_corr_data=None,
gmo_corr_data=gv_data,model_type=model_type)
print(gmo_direct)
fit_out = gmo_direct.get_fit()

{'gmo_E': array([0.002(10), 0.003(15)], dtype=object),'gmo_z_PS': array([0.7(7.0), 0.7(7.0)], dtype=object),'gmo_z_SS': array([0.7(7.0), 0.7(7.0)], dtype=object)}
Model Type:gmo_direct

	 N_{corr} = 2	
	 t_{corr} = [10, 25]
Fit results: 
Least Square Fit:
  chi2/dof [dof] = 2.9 [18]    Q = 2.6e-05    logGBF = 84.073

Parameters:
     gmo_z_PS 0   0.60559 (84)      [  0.7 (7.0) ]  
              1    -0.004 (15)      [  0.7 (7.0) ]  
     gmo_z_SS 0   0.60560 (91)      [  0.7 (7.0) ]  
              1     0.020 (17)      [  0.7 (7.0) ]  
         gmo_E0   0.00196 (16)      [ 0.002 (10) ]  
  gmo_log(dE) 0     -0.50 (40)      [    -7 (15) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08*,1e-10,1e-10)    (itns/time = 110/0.0)



In [5]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.colors as colors
from matplotlib.backends.backend_pdf import PdfPages
gmo_eff_mass = gmo_direct.get_gmo_effective(gmo_ratio=gv_data)
plot1 = gmo_direct.return_best_fit_info(bs=False)
plot2 = gmo_direct.plot_delta_gmo(correlators_gv=gv_data,t_plot_min=0,t_plot_max=20,
model_type=model_type,fig_name = None,show_fit=True)
plot3 = gmo_direct.plot_gmo_effective_mass(effective_mass=gmo_eff_mass,
t_plot_min=0,t_plot_max=40,model_type=model_type,show_fit=True,fig_name=None)
output_dir = 'fit_results/{0}/{1}_{0}'.format(p_dict['abbr'],model_type)
output_pdf = output_dir+".pdf"
with PdfPages(output_pdf) as pp:
    pp.savefig(plot1)
    pp.savefig(plot2)
    pp.savefig(plot3)

In [6]:
# print(temp)
data = {}
gmo = {}
for src_snk in ['PS', 'SS']:
    gmo['gmo_'+src_snk] = {}
    for baryon in['lambda_z', 'sigma_p', 'proton', 'xi_z']:
            data[baryon+'_'+src_snk] = 
for n in range(bs_N):
    for src_snk in ['PS', 'SS']:
        for baryon in ['lambda_z', 'sigma_p', 'proton', 'xi_z']:
            data[baryon+'_'+src_snk] = np.mean(temp[baryon+'_'+src_snk], axis=1,dtype=object)
            gmo['gmo_'+src_snk] = data['lambda_z_'+src_snk][n] *np.power(data['sigma_p_'+src_snk][n], 1/3) *np.power(data['proton_'+src_snk][n], -2/3) * np.power(data['xi_z_'+src_snk][n], -2/3)


SyntaxError: invalid syntax (76512496.py, line 7)

In [ ]:
gmo

# Samples_ps= gmo['gmo_PS']
# Samples_ss= gmo['gmo_SS']
# vals_ps = np.fromiter(Samples_ps.values(), dtype=float)
# vals_ss = np.fromiter(Samples_ss.values(), dtype=float)
# gv.dataset.avg_data(vals_ps)


{'gmo_PS': 0.5992642913502759, 'gmo_SS': 0.5979099635754775}